In [55]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from os import listdir
import random

# Mycobacterium tuberculosis H37Rv

In order to have the same RefSeq on both databases, this subspecie is used

In [138]:
bacteria = 'Mycobacterium'
root = 'E:/User/bruna.fistarol/Documents/GitHub'

# Data From PATRIC

## Genomic Features

The table below contains a list of genomic features, including coding DNA.

Each feature is solely identified by BRC ID and associated to a protein family referred as PATRIC genus-specific families (PLfams). Most of the genes has the associated RefSeq.

In [57]:
features = pd.read_csv('genome_features.csv')

Through this table, we extract useful data to map protein families referred by Nguyen et. al.:

In [58]:
plf = features[['BRC ID', 'PATRIC genus-specific families (PLfams)']].astype("string")
plf.columns = ['BRC_ID', 'PLFam']
plf.set_index('BRC_ID', inplace = True)
plf

,PLFam
BRC_ID,
fig|83332.12.peg.1000,PLF_1763_00001652
fig|83332.12.peg.1001,PLF_1763_00001934
fig|83332.12.peg.1002,PLF_1763_00021396
fig|83332.12.peg.1003,PLF_1763_00003246
fig|83332.12.peg.1004,PLF_1763_00003200
...,...
fig|83332.12.peg.7,PLF_1763_00003574
fig|83332.12.peg.997,PLF_1763_00002802
fig|83332.12.peg.104,PLF_1763_00002641


## Specialty Genes

The table containing specialty genes relates several genomic features to a relevant property. Here the table is filtered by the antibiotic resistance property

In [59]:
AMR_genes = pd.DataFrame(pd.read_csv('specialty_genes.csv')['BRC ID'])
AMR_genes.columns = ['BRC_ID']

## Data from Nguyen et. al.

In [60]:
plf_500 = []

datadir = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
for strain in listdir(datadir):
    with open(os.path.join(datadir, strain), 'r') as sequences:
        for line in sequences:
            if line[0] == '>':
                plf_500.append(line[1:len(line)-1])
                
plf_500 = pd.DataFrame(np.unique(plf_500))
plf_500.columns = ['Protein Family ID']

## RefSeq Mapping

In [61]:
refseq = features[['BRC ID', 'RefSeq Locus Tag']]
refseq.columns = ['BRC_ID', 'RefSeq']
refseq.set_index('BRC_ID', inplace = True)

In [62]:
AMR_refseq = features[['RefSeq Locus Tag']][features['BRC ID'].isin(AMR_genes['BRC_ID'])].reset_index(drop = True)
AMR_refseq.columns = ['AMR_RefSeq']

In [111]:
plf_map_refseq = features[['RefSeq Locus Tag', 'PATRIC genus-specific families (PLfams)']][features['PATRIC genus-specific families (PLfams)'].isin(plf_500['Protein Family ID'])].reset_index(drop = True)
plf_map_refseq.columns = ['RefSeq', 'PLF']
plf_map_refseq.dropna(inplace = True)
plf_map_refseq = plf_map_refseq.reset_index(drop = True)

In [112]:
plf_map_refseq

,RefSeq,PLF
0,Rv0903c,PLF_1763_00374132
1,Rv0917,PLF_1763_00002759
2,Rv0921,PLF_1763_00002626
3,Rv0926c,PLF_1763_00000707
4,Rv0930,PLF_1763_00001754
...,...,...
480,Rv0883c,PLF_1763_00195347
481,Rv0885,PLF_1763_00000149
482,Rv0887c,PLF_1763_00049773
483,Rv0892,PLF_1763_00009904


## Protein Interaction Network

In [65]:
ppi_patric = pd.read_csv('ppi_patric.csv')
ppi_patric = ppi_patric[['Interactor A ID', 'Interactor B ID']].astype("string")
ppi_patric.columns = ['Interactor_A_ID', 'Interactor_B_ID']

In [66]:
ppi_refseq = ppi_patric
for i in range(len(ppi_refseq['Interactor_A_ID'])):
    if ppi_refseq['Interactor_A_ID'][i] in list(refseq.index):
        ppi_refseq.at[i, 'Interactor_A_ID'] = refseq.loc[ppi_refseq['Interactor_A_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)
        
ppi_refseq.reset_index(inplace = True, drop = True)
        
for i in range(len(ppi_refseq['Interactor_B_ID'])):
    if ppi_refseq['Interactor_B_ID'][i]in (refseq.index):
        ppi_refseq.at[i, 'Interactor_B_ID'] = refseq.loc[ppi_refseq['Interactor_B_ID'][i]].RefSeq
    else:
        ppi_refseq.drop(inplace = True, labels = i)
        
ppi_refseq.reset_index(inplace = True, drop = True)

In [67]:
ppi_string = pd.read_csv('ppi_string.txt', sep = ' ')
ppi_string.columns = ['Interactor_A_ID', 'Interactor_B_ID', 'weight']
ppi_string.replace('83332.', '', regex = True, inplace = True)

In [72]:
ppi_all = pd.DataFrame(pd.concat([ppi_refseq, ppi_string], axis = 0))

In [73]:
ppi_all.fillna(999, inplace = True)

In [74]:
ppi_all

,Interactor_A_ID,Interactor_B_ID,weight
0,Rv3916c,Rv2011c,999.0
1,Rv3916c,Rv0805,999.0
2,Rv3916c,Rv2982c,999.0
3,Rv3717,Rv2723,999.0
4,Rv0307c,Rv0409,999.0
...,...,...,...
185801,Rv3924c,Rv0722,998.0
185802,Rv3924c,Rv0723,998.0
185803,Rv3924c,Rv1293,236.0
185804,Rv3924c,Rv1388,851.0


In [123]:
ppi = ppi_all[ppi_all['weight'] > 400].reset_index()

In [141]:
ppi

,index,Interactor_A_ID,Interactor_B_ID,weight
0,0,Rv3916c,Rv2011c,999.0
1,1,Rv3916c,Rv0805,999.0
2,2,Rv3916c,Rv2982c,999.0
3,3,Rv3717,Rv2723,999.0
4,4,Rv0307c,Rv0409,999.0
...,...,...,...,...
20249,185800,Rv3924c,Rv0634B,968.0
20250,185801,Rv3924c,Rv0722,998.0
20251,185802,Rv3924c,Rv0723,998.0
20252,185804,Rv3924c,Rv1388,851.0


In [124]:
conserved_ppi_A = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_A_ID'])]['RefSeq']
conserved_ppi_B = plf_map_refseq[plf_map_refseq['RefSeq'].isin(ppi['Interactor_B_ID'])]['RefSeq']

conserved_ppi = pd.DataFrame(pd.concat([conserved_ppi_A, conserved_ppi_B], axis = 0).drop_duplicates())

In [125]:
conserved_ppi

,RefSeq
0,Rv0903c
4,Rv0930
5,Rv0941c
6,Rv0944
8,Rv0948c
...,...
461,Rv0697
464,Rv0077c
469,Rv0799c
482,Rv0887c


In [126]:
AMR_ppi_A = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_A_ID'])]['AMR_RefSeq']
AMR_ppi_B = AMR_refseq[AMR_refseq['AMR_RefSeq'].isin(ppi['Interactor_B_ID'])]['AMR_RefSeq']

AMR_ppi = pd.DataFrame(pd.concat([AMR_ppi_A, AMR_ppi_B], axis = 0))
AMR_ppi.reset_index(drop=True, inplace=True)

In [127]:
AMR_ppi

,AMR_RefSeq
0,Rv1267c
1,Rv1297
2,Rv1315
3,Rv1484
4,Rv1536
...,...
75,Rv0682
76,Rv0684
77,Rv0685
78,Rv0700


In [128]:
ppi_info = pd.DataFrame(columns = ['Conserved Gene', 'Shortest Path to an AMR gene (length)',])

ppi_info['Conserved Gene'] = conserved_ppi.reset_index(drop = True)['RefSeq']

In [129]:
ppi_graph = nx.from_pandas_edgelist(ppi, 'Interactor_A_ID', 'Interactor_B_ID')

In [130]:
ppi_graph.number_of_edges()

10790

In [131]:
ppi_graph.number_of_nodes()

2176

In [132]:
idx = 0
for i in conserved_ppi['RefSeq']:
    lengths = []
    for j in AMR_ppi['AMR_RefSeq']:
        if nx.has_path(ppi_graph, i, j):
            lengths.append(nx.shortest_path_length(ppi_graph, i, j))
    if lengths:        
        ppi_info['Shortest Path to an AMR gene (length)'][idx] = min(lengths)
        
    idx += 1

In [133]:
print(ppi_info.groupby(['Shortest Path to an AMR gene (length)']).size().reset_index(name='Count'))

   Shortest Path to an AMR gene (length)  Count
0                                      1     28
1                                      2    176
2                                      3     43
3                                      4      6
4                                      5      7
5                                      6      1


In [134]:
plf_map_refseq = plf_map_refseq.set_index('RefSeq')

In [135]:
for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    gene_set = ppi_info[ppi_info['Shortest Path to an AMR gene (length)'] == i]['Conserved Gene']
    globals()[f'plf_length_{i}'] = plf_map_refseq.loc[list(gene_set)]['PLF']

In [136]:
bacdir = f'{root}/Fistarol_2022_2.0/{bacteria}'

os.mkdir(bacdir)

for i in ppi_info['Shortest Path to an AMR gene (length)'].unique():
    newdir = f'length.{i}'
    lendir = os.path.join(bacdir, newdir)
    os.mkdir(path)
    
    sample = f'{root}/Nguyen_et_al_2020/{bacteria}/fasta.500.0'
    for strain in listdir(sample):
        with open(os.path.join(lendir, strain), 'a') as mystrain:
            with open(os.path.join(sample, strain), 'r') as sequences:
                first_loop = True
                for line in sequences:
                    if line[0] == '>':
                        if first_loop:
                            plfam = line[1:len(line)-1]
                            seq = ''
                            first_loop = False
                            continue
                        if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)
                        plfam = line[1:len(line)-1]
                        seq = ''
                    else:
                        seq += line
                if plfam in list((globals()[f'plf_length_{i}'])):
                            mystrain.write('>' + plfam + '\n')
                            mystrain.write(seq)

At this point, it is possible to use this new configuration of data to run the model. We can take 25 genes to each strain for lengths equals to 1, 2 and 3, because the results from the paper also are derived from groups of 25 genes, hence, we can compare these results.

In [140]:
dir_25genes = os.path.join(bacdir, '25genes')
os.mkdir(dir_25genes)

for j in [0, 1, 2, 3, 4]:
    
    rand_idx = [sorted(random.sample(range(28), 25)), 
                sorted(random.sample(range(176), 25)), 
                sorted(random.sample(range(43), 25))]
    
    for i in range(1, len(rand_idx)+1): 
        
        lendir = os.path.join(bacdir, f'length.{i}')
        repdir = os.path.join(dir_25genes, f'length.{i}.{j}')
        os.mkdir(repdir)

        for strain in listdir(lendir):
            with open(os.path.join(repdir, strain), 'a') as mystrain:
                with open(os.path.join(lendir, strain), 'r') as sequences:
                    c = 0
                    first_loop = True
                    for line in sequences:
                        if line[0] == '>':
                            if first_loop:
                                plfam = line
                                seq = ''
                                c += 1
                                first_loop = False
                                continue
                            if c in rand_idx[i-1]:
                                mystrain.write(plfam)
                                mystrain.write(seq)
                            plfam = line
                            seq = ''
                            c += 1
                        else:
                            seq += line
                    if c in rand_idx[i-1]:
                            mystrain.write(plfam)
                            mystrain.write(seq)